In [1]:
import math
from enum import Enum

## Calculating AQI from EPA

## What is the U.S. Air Quality Index - AQI?
Read more [README.md](../docs/README.md)

## How do I Calculate the AQI from pollutant concentration data?

The AQI is the highest value calculated for each pollutan as follows:

    A. Identify the highest concentration among all of the monitors within each reporting area and truncate as follow:

![Pollutants](../docs/pollutants.png)

    B. Using Table 5, find the two breakpoints tha contain the concentration.

![Pollutants Breakpoints](../docs/pollutants-breakpoints.png)

    C. Using Equation 1, calculate the index

![Equation 1](../docs/equation1.png)

    D. Round the index to the nearest integer


## AQI Levels
![AQI Levels](../docs/aqi-epa.png)

### AQI and PM2.5

### Contants

In [2]:
# AQI Max and Min Values
AQI_MIN_VALUE=0
AQI_MAX_VALUE=500
# PM2.5 Max and Min Values
PM25_MIN_VALUE=0.0
PM25_MAX_VALUE=500.4

In [3]:
class RangeValues:
    min: int | float
    max: int | float

    def __init__(self, min: int | float=0, max: int | float=0):
        self.min = min
        self.max = max
    
    def is_in(self, value: int | float) -> bool:
        if value >= self.min and value <= self.max:
            return True
        return False
    
    def __str__(self) -> str:
        return f"Min: {self.min}, Max: {self.max}"


class MeasureInfo:
    title: str
    desc: str
    color: str

    def __init__(self, title: str, desc: str, color: str):
        self.title = title
        self.desc = desc
        self.color = color
    
    def __str__(self) -> str:
        return f"Title: {self.title}, Description: {self.desc}, Color: {self.color}"


class MeasureLevels(Enum):

    GOOD: MeasureInfo = MeasureInfo(
        "Good", 
        "Air quality is satisfactory, and air pollution poses little or no risk", 
        "#00e400"
    )
    MODERATE: MeasureInfo = MeasureInfo(
        "Moderate", 
        "Air quality is acceptable. However, there may be a risk for some people, particularly those who are unusually sensitive to air pollution", 
        "#ffff00"
    )
    UNHEALTHY_FOR_SENSITIVE_GROUPS: MeasureInfo = MeasureInfo(
        "Unhealthy for Sensitive Groups", 
        "Members of sensitive groups may experience health effects. The general public is less likely to be affected", 
        "#ff7e00"
    )
    UNHEALTHY: MeasureInfo = MeasureInfo(
        "Unhealthy", 
        "Some members of the general public may experience health effects; members of sensitive groups may experience more serious health effects", 
        "#ff0000"
    )
    VERY_UNHEALTHY: MeasureInfo = MeasureInfo(
        "Very Unhealthy", 
        "Health alert: The risk of health effects is increased for everyone", 
        "#8f3f97"
    )
    HAZARDOUS: MeasureInfo = MeasureInfo(
        "Hazardous", 
        "Health warning of emergency conditions: everyone is more likely to be affected", 
        "#7e0023"
    )


class IndexEPA():
    measure_level: MeasureLevels
    range: RangeValues

    def __init__(self, measure_level: MeasureLevels, range: RangeValues):
        self. measure_level = measure_level
        self.range = range
    
    def __str__(self) -> str:
        return f"Measure Level: {self.measure_level}, Range Values: {self.range}"


class AQILevels(Enum):

    GOOD: IndexEPA = IndexEPA(MeasureLevels.GOOD, RangeValues(AQI_MIN_VALUE, 50))
    MODERATE: IndexEPA = IndexEPA(MeasureLevels.MODERATE, RangeValues(51, 100))
    UNHEALTHY_FOR_SENSITIVE_GROUPS: IndexEPA = IndexEPA(MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, RangeValues(101, 150))
    UNHEALTHY: IndexEPA = IndexEPA(MeasureLevels.UNHEALTHY, RangeValues(151, 200))
    VERY_UNHEALTHY: IndexEPA = IndexEPA(MeasureLevels.VERY_UNHEALTHY, RangeValues(201, 300))
    HAZARDOUS: IndexEPA = IndexEPA(MeasureLevels.HAZARDOUS, RangeValues(301, AQI_MAX_VALUE))


class PM25Levels(Enum):

    GOOD: IndexEPA = IndexEPA(MeasureLevels.GOOD, RangeValues(PM25_MIN_VALUE, 12.0))
    MODERATE: IndexEPA = IndexEPA(MeasureLevels.MODERATE, RangeValues(12.1, 35.4))
    UNHEALTHY_FOR_SENSITIVE_GROUPS: IndexEPA = IndexEPA(MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, RangeValues(35.5, 55.4))
    UNHEALTHY: IndexEPA = IndexEPA(MeasureLevels.UNHEALTHY, RangeValues(55.5, 150.4))
    VERY_UNHEALTHY: IndexEPA = IndexEPA(MeasureLevels.VERY_UNHEALTHY, RangeValues(150.5, 250.4))
    HAZARDOUS: IndexEPA = IndexEPA(MeasureLevels.HAZARDOUS, RangeValues(250.5, PM25_MAX_VALUE))


def get_index_epa(value: int | float, measure: Enum) -> IndexEPA:
    for index in measure:
        if index.value.range.is_in(value):
            return index.value


### Equation 1

In [4]:
# Define Equation 1 to Calculate PM2.5 AQI Value
#
# AQI = AQI for pollutant p
# Cp = The truncated concentrattion of pollutant p, This cases PM2.5
# BP_hi = The concentration breakpoint that is greater than or equal to Cp
# BP_lo = The concentration breakpoint that is less than or equal to Cp
# AQI_hi = The AQI value corresponding to BP_hi
# AQI_lo = The AQI value corresponding to BP_lo
#

def pm25_to_aqi(pm25_value: float) -> int:
    # Truncate PM2.5 Value
    pm25_value = float(f'{pm25_value:.1f}')
    # AQI Value to Concentration of PM2.5
    aqi_value = None
    # Variables
    BP_hi = None # The concentration breakpoint that is greater than or equal to pm25_value
    BP_lo = None # The concentration breakpoint that is less than or equal to pm25_value
    AQI_hi = None # The AQI value corresponding to BP_hi
    AQI_lo = None # The AQI value corresponding to BP_lo

    # Get BP_hi and BP_lo
    for bp in PM25Levels:
        #print('bp:', bp)
        if bp.value.range.is_in(pm25_value):
            BP_hi = bp.value.range.max
            BP_lo = bp.value.range.min
            AQI_hi = AQILevels[bp.name].value.range.max
            AQI_lo = AQILevels[bp.name].value.range.min
            break

    aqi_value = pm25_value if math.isnan(pm25_value) else math.ceil((((AQI_hi - AQI_lo) / (BP_hi - BP_lo)) * (pm25_value - BP_lo)) + AQI_lo)

    return aqi_value

In [5]:
# Example:
# Equation 1 to calculate the AQI value
pm25_value = 35.9
#pm25_value = math.nan
aqi_value = pm25_to_aqi(pm25_value)

print(f"PM2.5: {pm25_value}, AQI: {aqi_value}")
print(f"PM2.5: {pm25_value}, {get_index_epa(pm25_value, PM25Levels)}")
print(f"AQI: {aqi_value}, {get_index_epa(aqi_value, AQILevels)}")

PM2.5: 35.9, AQI: 102
PM2.5: 35.9, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 35.5, Max: 55.4
AQI: 102, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 101, Max: 150
